# Loading Label Files

In [1]:
# Dataset loading
import os

dataset = "./Dataset (Labelled Images)/"
lab_files_path = dataset+"label/"

files = os.listdir(lab_files_path)
print("Total Label files are: ", len(files))

Total Label files are:  129


# Sparse Matrix Creation

Iterate all the csv files and do the following three steps:
1. Make a graph of CSV file.
2. Turn the graph into adjacency matrix.
3. Append that adjacency matrix into sparce matrix(i.e. bigger matrix)

# DON'T RUN THIS TAKES A HELL LOT OF TIME
# SPARCE MATRIX IS ALREADY CREATED AND SAVED

In [4]:
import pandas as pd
import Mat_Package.Grapher as GMaker
import Mat_Package.MatricesOverDiagonal as DiagPlacer
import networkx as nx
import numpy as np
from numpy.core.fromnumeric import shape


Z_file = lab_files_path + files[0]
df = pd.read_csv(Z_file)
G = GMaker.makeGraph(df)
M = nx.to_numpy_array(G, dtype=np.int32)


for i in range(1, len(files)):
    print("\n= = = = = = = = = = = = = = = =")
    print("Iteration No.: ", i)
    # Getting file
    f = lab_files_path + files[i]
    # Making dataframe of file
    df = pd.read_csv(f)
    # Making graph of the dataframe. 
    G = GMaker.makeGraph(df)
    # Storing the graph as an incident matrix(an adjacency matrix)
    I = nx.to_numpy_array(G, dtype=np.int32)
    # Now resizing the original sparce matrix with the new incident matrix
    M = DiagPlacer.resizeMatrix(M, I)
    
print("Final Matix Done")
print("Dimentions of sparce matrix are: ", np.shape(M))



= = = = = = = = = = = = = = = =
Iteration No.:  1

= = = = = = = = = = = = = = = =
Iteration No.:  2

= = = = = = = = = = = = = = = =
Iteration No.:  3

= = = = = = = = = = = = = = = =
Iteration No.:  4

= = = = = = = = = = = = = = = =
Iteration No.:  5

= = = = = = = = = = = = = = = =
Iteration No.:  6

= = = = = = = = = = = = = = = =
Iteration No.:  7

= = = = = = = = = = = = = = = =
Iteration No.:  8

= = = = = = = = = = = = = = = =
Iteration No.:  9

= = = = = = = = = = = = = = = =
Iteration No.:  10

= = = = = = = = = = = = = = = =
Iteration No.:  11

= = = = = = = = = = = = = = = =
Iteration No.:  12

= = = = = = = = = = = = = = = =
Iteration No.:  13

= = = = = = = = = = = = = = = =
Iteration No.:  14

= = = = = = = = = = = = = = = =
Iteration No.:  15

= = = = = = = = = = = = = = = =
Iteration No.:  16

= = = = = = = = = = = = = = = =
Iteration No.:  17

= = = = = = = = = = = = = = = =
Iteration No.:  18

= = = = = = = = = = = = = = = =
Iteration No.:  19

= = = = = = = = = = 

# Saving Matrix

# MATRIX ALREADY SAVED
# PLEASE DONOT RUN THIS CELL

In [5]:
print("Saving Matrix")
np.save("Matrix.npy", M)

print("Saved")

Saving Matrix
Saved


# Loading Matrix
Loading the sparce matrix in a variable A.

In [2]:

import numpy as np

# loading matrix
A = np.load("Matrix.npy")
print("Printing Sparse Matrix..")
print(A)
print(np.shape(A))

Printing Sparse Matrix..
[[1 1 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]
(21097, 21097)


# Generating and Encoding Labels
1. Get a list of labels from CSV files.
2. Catagorically encode the labels 

In [3]:
import Mat_Package.labels as lbls
from sklearn.preprocessing import LabelEncoder

labels = lbls.returnLabels(lab_files_path, files)
print("Total labels are: ", len(labels))

print("Encoding list..")
lab_encoder = LabelEncoder()
encodings = lab_encoder.fit_transform(labels)
print("Total Encodings are: ", len(encodings))


t_file = open("Encodings.txt", "w")
for element in encodings:
    t_file.write("%i\n" % element)

t_file.close()


uni_enc = set(encodings)
print("Unique Encodings are: ", uni_enc)

Total labels are:  21097
Encoding list..
Total Encodings are:  21097
Unique Encodings are:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


# Importing Libraries

In [7]:
from sklearn.utils import shuffle

# from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2


from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.layers import GCNConv
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
print("Libraries Imported..")


Libraries Imported..


# Model Hyper-parameters
Adding model hyper-parameters:
1. Learning rate(At whar pace should the model be learning)
2. Channels(Neurons)
3. Dropout rate(Now many neurons should be active at a time)
4. Epochs(How many times a data is passed through the model)
5. Batch size(How much data should be fed to the model at a time)
etc

In [8]:
# Hyper-parameters
channels = 8
dropout = 0.5
l2_reg = 5e-4
learning_rate = 1e-2
epochs = 200
batch_size = 1
es_patience = 10

# Model Definition
3 GCN-Conv Layers & 2 Dense Layers

In [9]:
X_in = Input(shape=np.shape(A)[0], )
fltr_in = Input((np.shape(A)[0]), sparse = True)

D1 = Dropout(dropout)(X_in)
G1 = GCNConv(channels, activation='relu', kernel_regularizer=l2(l2_reg), use_bias=True)([D1, fltr_in])
D2 = Dropout(dropout)(G1)
G2 = GCNConv(channels, activation='relu', kernel_regularizer=l2(l2_reg), use_bias=True)([D2, fltr_in])

flatten = Flatten()(G2)
D1 = Dense(8, activation='relu')(flatten)
D2_out = Dense(4, activation='softmax')(D1)

model = Model(inputs = [X_in, fltr_in], outputs=D2_out)
model.compile(optimizer = 'Adam', loss = 'catagorical_crossentropy', weighted_metrics=['acc'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 21097)]      0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 21097)        0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 21097)]      0                                            
__________________________________________________________________________________________________
gcn_conv (GCNConv)              (None, 8)            168784      dropout[0][0]                    
                                                                 input_2[0][0]                

# Training Model

In [10]:

# Feature Matrix
X = np.matrix([[i, -i] for i in range(np.shape(A)[0])], dtype=np.int32)
# print(X)

model.fit([X, A],
          encodings,
          epochs = epochs,
          batch_size = batch_size,
          verbose = 1
          )

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.